# Concentrations and the Age of Galaxy Clusters

This notebook is an application of the xlensing code over several surveys to investigate the relationship between cluster mass concentration and observables that may relate to mass accretion age.

In [1]:
import numpy as np
np.set_printoptions(linewidth=400)
from matplotlib import pyplot as plt
from getdist import plots, MCSamples
from astropy.table import Table

import os
import pickle

import xlensing

import emcee
import time

Lookup table loaded!


## Fetching Data

CS82: A weak-lensing survey done on stripe 82 by CFHT

In [2]:
CS82_DIR = os.path.expanduser("~/Data/CS82_REDMAPPER")
CS82_SOURCES = "sources_cleaner_final.fits"
CS82_CLUSTERS = "tcpure.fits"

In [3]:
CS82_SOURCES

'sources_cleaner_final.fits'

DES: A multi probe modern survey on the south hemisphere

In [4]:
DES_DIR = "Data/DES_REDMAPPER"
DES_SOURCES = "sources_cleaner_final.fits"
DES_CLUSTERS = "tcpure.fits"

## CS82 Data

In [5]:
sources  = Table.read(os.path.join(CS82_DIR,CS82_SOURCES))
clusters = Table.read(os.path.join(CS82_DIR,CS82_CLUSTERS))

In [6]:
#CS82_SOURCES2 = "cs82_combined_lensfit_sources_nozcuts_aug_2015.fits"

In [7]:
sources

RA,DEC,BPZ_ZPHOT,BPZ_LOW95,BPZ_ODDS,WEIGHT,M,E1CORR,E2CORR
float64,float64,float32,float32,float32,float32,float32,float32,float32
-24.69918173058005,-1.0244722207927224,1.25,0.1,0.40808,13.9551,-0.034325,0.13697599,0.236063
-25.11671359217877,-1.0229941001552796,0.38,0.23,0.95527,15.4969,-0.000684,0.027918,-0.161017
-25.113486639321934,-1.0234552707089264,0.66,0.48,0.87132,15.3687,-0.000158,-0.148118,0.137512
-25.108944123579704,-1.0215600239686817,0.48,0.15,0.61343,11.1226,-0.032525,-0.262904,0.700844
-25.210098782545572,-1.0212686890624931,0.87,0.15,0.54222,12.5195,-0.026663,-0.114856,0.22005899
-24.737692704477354,-1.0213363942721754,0.32,0.191,0.99715,15.2731,-0.001205,0.313868,-0.109652005
-25.04084649690043,-1.021474675107365,0.31,0.08,0.58817,14.2494,-0.004033,-0.224612,-0.43318102
-24.496686640601865,-1.0211324445396226,0.4,0.15,0.69965,9.7677,-0.099137,-0.090606995,-0.094464995
-24.587590676205878,-1.0211156088397502,0.86,0.51,0.73007,14.2706,-0.019731,0.176663,-0.318876


In [8]:
clusters['RA_RAD'] = np.radians(clusters['RA'])
clusters['DEC_RAD']= np.radians(clusters['DEC'])
clusters['loc'] = np.array([i for i in range(len(clusters))])

sr_RA = np.radians(np.array(sources['RA']))
sr_DEC= np.radians(np.array(sources['DEC']))
sr_z  = np.array(sources['BPZ_ZPHOT'])
#sr_z95low= np.array(sources2['BPZ_LOW95'])
sr_E1 = np.array(sources['E1CORR'])
sr_E2 = np.array(sources['E2CORR'])
sr_W = np.array(sources['WEIGHT'])
sr_M = np.array(sources['M'])

clusters['INDEX'] = np.array(range(len(clusters)))

In [9]:
cl_RA=np.radians(np.array(clusters['RA']))
cl_DEC= np.radians(np.array(clusters['DEC']))
cl_z= np.array(clusters['Z'])
cl = np.array([cl_RA,cl_DEC,cl_z]).T

Getting lensing signal for all clusters:

In [10]:
#let's use multiprocessing
from multiprocessing import Pool, freeze_support, cpu_count
from functools import partial

# you can use whatever, but your machine core count is usually a good choice (although maybe not the best)
pool = Pool(8) 


#We get a partial function with a constant galaxy catalogue to iterate with clusters.
survey_lensing = partial(xlensing.data.cluster_lensing,sources=(sr_RA, sr_DEC, sr_z, sr_E1, sr_E2, sr_W,sr_M),radius=10.)

In [11]:
#Make a list of clusters to get lensing data
clz = zip(cl_RA,cl_DEC,cl_z)
clzlist = [x for x in clz]

In [12]:
results = pool.map(survey_lensing, clzlist)

In [13]:
#pickle_out = open("Results/CS82/CS82_lensing_results.pickle","wb")
#pickle.dump(results, pickle_out)
#pickle_out.close()

### Stacking CS82 Data

In [14]:
radii = np.logspace(-.8,0.8,20)
N = len(radii)
bins_lims = np.logspace(np.log10(radii[0])+(np.log10(radii[0])-np.log10(radii[1]))/2,
                        np.log10(radii[N-1])-(np.log10(radii[0])-np.log10(radii[1]))/2,N+1)
bins_lims = np.array([[bins_lims[i],bins_lims[i+1]] for i in range(N)])
bins_lims #in Mpc/h

array([[0.14384499, 0.17462454],
       [0.17462454, 0.2119902 ],
       [0.2119902 , 0.25735127],
       [0.25735127, 0.31241857],
       [0.31241857, 0.37926902],
       [0.37926902, 0.46042394],
       [0.46042394, 0.55894416],
       [0.55894416, 0.67854546],
       [0.67854546, 0.82373871],
       [0.82373871, 1.        ],
       [1.        , 1.21397719],
       [1.21397719, 1.47374062],
       [1.47374062, 1.78908749],
       [1.78908749, 2.1719114 ],
       [2.1719114 , 2.6366509 ],
       [2.6366509 , 3.20083405],
       [3.20083405, 3.88573952],
       [3.88573952, 4.71719914],
       [4.71719914, 5.72657215],
       [5.72657215, 6.95192796]])

In [17]:
clusters.colnames

['MEM_MATCH_ID',
 'RA',
 'DEC',
 'MODEL_MAG',
 'MODEL_MAGERR',
 'IMAG',
 'IMAG_ERR',
 'ZRED',
 'ZRED_E',
 'BCG_SPEC_Z',
 'Z_SPEC_INIT',
 'Z_INIT',
 'Z',
 'LAMBDA_CHISQ',
 'LAMBDA_CHISQ_E',
 'R_LAMBDA',
 'SCALEVAL',
 'MASKFRAC',
 'C_LAMBDA',
 'C_LAMBDA_ERR',
 'MAG_LAMBDA_ERR',
 'Z_LAMBDA',
 'Z_LAMBDA_E',
 'PHOTOID',
 'LNLAMLIKE',
 'LNBCGLIKE',
 'LNLIKE',
 'PZBINS',
 'PZ',
 'NCROSS',
 'CHISQ',
 'RMASK',
 'RA_ORIG',
 'DEC_ORIG',
 'W',
 'DLAMBDA_DZ',
 'DLAMBDA_DZ2',
 'LAMBDA_CHISQ_C',
 'LAMBDA_CHISQ_CE',
 'NCENT',
 'NCENT_GOOD',
 'RA_CENT',
 'DEC_CENT',
 'LAMBDA_CHISQ_CENT',
 'P_BCG',
 'P_CEN',
 'Q_CEN',
 'P_FG',
 'Q_MISS',
 'P_SAT',
 'P_C',
 'BCG_ILUM',
 'ILUM',
 'Z_LAMBDA_RAW',
 'Z_LAMBDA_E_RAW',
 'RMAG',
 'RMAG_ERR',
 'I_ABS',
 'R_ABS',
 'I_ABS_ERR',
 'RC',
 'RA_RAD',
 'DEC_RAD',
 'ANG_DIST',
 'OFF',
 'DELTA12',
 'DELTA14',
 'DELTA12_E',
 'DELTA14_E',
 'DELTA12i',
 'DELTA14i',
 'DELTA12i_E',
 'DELTA14i_E',
 'loc',
 'INDEX']

In [18]:
conditions =           (clusters['LAMBDA_CHISQ']<60)&\
                      (clusters['LAMBDA_CHISQ']>20)

clusters_lowz = clusters[(clusters['Z']>0.2)&
                         (clusters['Z']<0.4)&
                         conditions]

cuts = 3
observable = 'DELTA12'
clusters_lowz.sort(observable)
partitions = [x*100/cuts for x in range(1,cuts) ]

split = list(np.percentile(np.array(range(len(clusters_lowz))),partitions).astype(int))
lowz=np.split(np.array(clusters_lowz),split)

stick = []
for stake in lowz:
    print("# of clusters in stack: {}".format(len(stake)))
    print("mean, lowest, highest, redMaPPer richness: ",np.mean(stake['LAMBDA_CHISQ']),min(stake['LAMBDA_CHISQ']),max(stake['LAMBDA_CHISQ']))
    stick.append(Table(stake))

# of clusters in stack: 42
mean, lowest, highest, redMaPPer richness:  28.5608 20.38486 53.22882
# of clusters in stack: 42
mean, lowest, highest, redMaPPer richness:  29.64899 20.14078 59.02571
# of clusters in stack: 43
mean, lowest, highest, redMaPPer richness:  29.999043 20.14078 56.310326


In [ ]:
Nboot=500
print(bins_lims)
stick_results = []
for stake in stick:
    t = time.time()
    clusterbkgs = []
    for index in stake['INDEX']:
        Sigma_crit = np.array(results[index]['Critical Density'])
        e_t = np.array(results[index]['Tangential Shear'])
        e_x = np.array(results[index]['Critical Density'])
        W = np.array(results[index]['Weights'])
        M = np.array(results[index]['Mult. Bias'])
        R = np.array(results[index]['Radial Distance'])
        clusterbkgs.append(np.array([Sigma_crit, e_t, e_x, W, R,M]))
    sigmas, sigmas_cov, xigmas, xigmas_cov = xlensing.data.stack(clusterbkgs,bins_lims,Nboot)
    stick_results.append( ( sigmas, sigmas_cov, xigmas, xigmas_cov) )
    print("Done in " + str(time.time()-t) + " seconds.")

#pickle_out2 = open("Results/CS82/CS82_stacking_results.pickle","wb")
#pickle.dump(stick_results, pickle_out2)
#pickle_out2.close()

### Model

In [ ]:
def NFWsimple(theta,Z,radii):
    M200, C200, PCC, SIGMAOFF  = theta
    result = xlensing.model.NFW_shear(M200, C200, Z, PCC, SIGMAOFF, 0 ,radii) #+  xlensing.model.NFW_shear(M200, C200, Z, PCC, 0.65, 3.2e11,radii)['Miscentered Signal']
    
    return result['Signal'] #+ result['Miscentered Signal'] #result['Signal'] #

Priors:

In [ ]:
M200lo, M200hi = 1e13, 1e15
C200lo, C200hi = 0, 20
PCClo, PCChi = 0., 1
SIGMAOFFlo, SIGMAOFFhi = 0.01, 0.1
M0lo, M0hi = 0, 1e12

priorM200 = xlensing.fitting.ln_flat_prior_maker(M200lo, M200hi,0)
priorC200 = xlensing.fitting.ln_flat_prior_maker(C200lo, C200hi,1)
priorPCC =xlensing.fitting.ln_gaussian_prior_maker(0.75, 0.08,2) ##Zhang et al. 2019
priorSIGMAOFF = xlensing.fitting.ln_flat_prior_maker(SIGMAOFFlo, SIGMAOFFhi ,3)

#priorSIGMAOFF = xlensing.fitting.ln_gaussian_prior_maker(0.42, 0.002,3)
#priorM0       = xlensing.fitting.ln_flat_prior_maker(M0lo, M0hi,4)
prior = lambda theta : priorM200(theta) + priorC200(theta) + priorPCC(theta) +priorSIGMAOFF(theta) # + priorM0(theta)

### MCMC

In [ ]:
ndim, nwalkers, steps = 4, 32, 4*512
print("N/50 ={}".format(steps/50))
samplestick = []
samplerstick= []
#for each stack, run MCMC
burnin=round(steps/4.)
for stickresult in stick_results:
    mean_z = np.average(stake['Z'])
    #build data likelihood
    model = lambda theta: NFWsimple(theta,mean_z,radii)
    likelihood = xlensing.fitting.ln_gaussian_likelihood_maker((stickresult[0],stickresult[1]),model)
    posterior = lambda theta : likelihood(theta) +prior(theta)

    #initialise walkers
    pos = []
    for i in range(nwalkers):
        M200 = np.random.uniform(M200lo,M200hi)
        C200 = np.random.uniform(C200lo,C200hi)
        PCC  = np.random.uniform(PCClo,PCChi)
        SIGMAOFF = np.random.uniform(SIGMAOFFlo, SIGMAOFFhi)
        #M0       = np.random.uniform(M0lo, M0hi)
        pos.append(np.array([M200,C200,PCC,SIGMAOFF]))

    sampler = emcee.EnsembleSampler(nwalkers, ndim, posterior)
    print("Running MCMC...")
    t = time.time()
    sampler.run_mcmc(pos, steps, rstate0=np.random.get_state(),progress=True)
    print("Done in " + str(time.time()-t) + " seconds.")
    print(sampler.get_autocorr_time(tol=0))
    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
    samplestick.append(samples)
    samplerstick.append(sampler)

In [ ]:
for samplers in samplerstick:
    print("N/50 ={}".format(steps/50))
    print(samplers.get_autocorr_time(tol=0))

In [ ]:
#pickle_out = open("Results/CS82/chains.pickle","wb")
#pickle.dump(samplestick, pickle_out)
#pickle_out.close()

In [ ]:
for samples in samplestick:
    mvir_tru, conc_tru, pcc_tru, sigma_tru = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84],axis=0)))
    print("Mvir: {:.2e}".format(mvir_tru[0]) + " p {:.2e}".format(mvir_tru[1]) + " m {:.2e}".format(mvir_tru[2]))
    print("Conc: {:.2f}".format(conc_tru[0]) + " p {:.2f}".format(conc_tru[1]) + " m {:.2f}".format(conc_tru[2]))
    print("Pcc: {:.2f}".format(pcc_tru[0]) + " p {:.2f}".format(pcc_tru[1]) + " m {:.2f}".format(pcc_tru[2]))
    print("sigma_off: {:.2f}".format(sigma_tru[0]) + " p {:.2f}".format(sigma_tru[1]) + " m {:.2f}".format(sigma_tru[2]))
    #print("M0: {:.2e}".format(m0_tru[0]) + " p {:.2e}".format(m0_tru[1]) + " m {:.2e}".format(m0_tru[2]))
    print()

labs =  ["M_{200} [M_\\odot]", "c_{200}","p_{cc}","\\sigma_{off}"]#,"M_0"]

g = plots.getSubplotPlotter(width_inch=20)
sample = [MCSamples(samples=samples, names = labs,labels=labs) for samples in samplestick]
g.triangle_plot(sample,color=['r','g','b'],filled=True)
#g.subplots[0,0].set_xlim(1e13,3e14)
plt.savefig("triangle_plot_"+observable+ "_{}.pdf".format(time.time()))

In [ ]:
plt.figure(figsize=(7,9))
color = ['r','g','b']
labels = [observable + " " + str(i) for i in range(cuts)]
for i,stickresult in enumerate(stick_results):
    plt.scatter(radii,np.array(stickresult[0]), c=color[i], label = labels[i])
    plt.errorbar(radii,np.array(stickresult[0]),yerr=np.sqrt(np.diag(stickresult[1])),fmt='.',color=color[i])
for i,samples in enumerate(samplestick):
    mvir_tru,conc_tru,pcc_tru,sigma_tru = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84],axis=0)))
    theta = mvir_tru[0], conc_tru[0],pcc_tru[0],sigma_tru[0]#,0
    fitcurve = model(theta)
    plt.plot(radii,fitcurve,color=color[i])


plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e0,1e3)
plt.savefig('data_fit_'+observable+'_{}.pdf'.format(time.time()))

## Building new likelihoods

Single cluster:

## Multi Likelihood

The plan!

Find a concentration - offset scaling relation like mass/lambda

1. get a pivot/offset slope

2. use a mass/lambda slope (redMaPPer)

3. using  2, gets masses

4. using 1 & 3 + m-C relation gets C - Diemer, Joyce

5. NFW likelihood for each

6. sum all likelihoods


Input: 


In [ ]:
def conc_age(age,pivot,slope):
    conc = pivot*age**slope
    return conc

In [ ]:
"""Calculate halo concentration from mass and redshift.
"""

def c_DuttonMaccio(z, m):
    """Concentration from c(M) relation in Dutton & Maccio (2014).
    Parameters
    ----------
    z : float or array_like
        Redshift(s) of halos.
    m : float or array_like
        Mass(es) of halos (m200 definition), in units of solar masses.
    h : float, optional
        Hubble parameter. Default is from Planck13.
    Returns
    ----------
    ndarray
        Concentration values (c200) for halos.
    References
    ----------
    Calculation from Planck-based results of simulations presented in:
    A.A. Dutton & A.V. Maccio, "Cold dark matter haloes in the Planck era:
    evolution of structural parameters for Einasto and NFW profiles,"
    Monthly Notices of the Royal Astronomical Society, Volume 441, Issue 4,
    p.3359-3374, 2014.
    """


    a = 0.52 + 0.385 * np.exp(-0.617 * (z**1.21))  # EQ 10
    b = -0.101 + 0.026 * z                         # EQ 11

    logc200 = a + b * np.log10(m * 1 / (10.**12))  # EQ 7

    concentration = 10.**logc200

    return concentration

In [ ]:
c_DuttonMaccio(0.3,1e14)

In [ ]:
def mass_lambda_MEP(Lambda):
    Lambda0 = 40
    M0 = 2.21E14
    alpha = 1.18
    
    mass = M0*(Lambda/Lambda0)**alpha
    
    return mass

In [ ]:
"{:.2e}".format(mass_lambda_MEP(32))

In [ ]:
import colossus.halo

In [ ]:
from colossus.cosmology import cosmology
from colossus.halo import concentration

cosmology.setCosmology('planck15')

In [ ]:
concentration.concentration(1e14, '200c', 0.0, model = 'diemer15')

In [ ]:
def newmodel(richness, age, Z, radii, theta):
    
    PCC, SIGMAOFF, pivot_conc_age, slope_conc_age = theta
    
    M200 = mass_lambda_MEP(richness)
    C =  c_DuttonMaccio(Z,M200)
    C200 = C*conc_age(age,pivot_conc_age, slope_conc_age)
    
    
    result =xlensing.model.NFW_shear(M200, C200, Z, PCC, SIGMAOFF, 0 ,radii)['Signal']
    
    return result, M200, C, C200

In [ ]:
theta = (0.7, 0.4, 1,0.1)

In [ ]:
newmodel(40, 2 , 0.3, radii, theta)